# Random Flow


Simulate trajectories of a particle cloud in a randomly generated flow field.
A doubly periodic domain is used and an animation generated.
For additional documentation e.g. see :
[1](https://JuliaClimate.github.io/IndividualDisplacements.jl/dev/),
[2](https://JuliaClimate.github.io/MeshArrays.jl/dev/),
[3](https://docs.juliadiffeq.org/latest/solvers/ode_solve.html),
[4](https://en.wikipedia.org/wiki/Displacement_(vector))

![particles in random flow](https://github.com/JuliaClimate/IndividualDisplacements.jl/raw/master/examples/figs/RandomFlow.gif)

## 1.1 Import Software

In [1]:
using OrdinaryDiffEq, IndividualDisplacements, MeshArrays
p=dirname(pathof(MeshArrays)); include(joinpath(p,"../examples/Demos.jl"))
p=dirname(pathof(IndividualDisplacements))
include(joinpath(p,"../examples/helper_functions.jl"))
include(joinpath(p,"../examples/recipes_plots.jl"));

## 1.2 Setup Problem

Put grid variables in a dictionary.

In [2]:
np=8
nq=12
Γ=simple_periodic_domain(np,nq);

Derive flow field from randomly generated ϕ streamfunction

In [3]:
𝑃,ϕ=setup_random_flow(Γ);

## 1.3 Initial Conditions

In [4]:
x0,x1=np .*(0.4,0.6)
y0,y1=np .*(0.4,0.6)

n=100
x0=x0 .+(x1-x0).*rand(n)
y0=y0 .+(y1-y0).*rand(n)

xy=transpose([x0[:] y0[:] ones(size(x0[:]))]);

## 2.1 Compute Trajectories

In [5]:
tr = DataFrame(ID=Int[], x=Float64[], y=Float64[], t=Float64[])
solv(prob) = solve(prob,Tsit5(),reltol=1e-5,abstol=1e-5)
𝐼 = Individuals{Float64}(📌=xy[:,:], 🔴=tr, 🆔=collect(1:size(xy,2)),
                         🚄 = dxy_dt!, ∫ = solv, 🔧 = postprocess_xy, 𝑃=𝑃)

𝑇=(0.0,𝐼.𝑃.𝑇[2])
∫!(𝐼,𝑇)

3×100 Array{Float64,2}:
 7.15222  5.29671  7.0969   6.10973  …  7.09505  6.23748  6.47122  6.60283
 3.49883  3.84662  3.41935  3.91619     3.86621  2.99488  3.60695  2.92859
 1.0      1.0      1.0      1.0         1.0      1.0      1.0      1.0

## 2.2 Plot Results

For example, generate a simple animation (with `if true`):

In [6]:
if false
anim = @animate for t in 0:2.0:maximum(𝐼.tr.t)
   phi_and_subset(Γ,ϕ,𝐼.tr,t)
end
pth=tempdir()*"/"
gif(anim, pth*"RandomFlow.gif", fps = 15)
end

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*